In [ ]:
# We shall import PREMISE here and see if we can develop prospective versions of ecoinvent for the ceramics process

In [ ]:
from premise import *
import bw2data

In [ ]:
# basic imports from brightway
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
from bw2io.importers import SingleOutputEcospold2Importer
import bw2analyzer as bwa
from bw2data import methods

# other relevant packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns

In [ ]:
# define a project where we install the databases and work in this script
bd.projects.set_current('brightway25')

In [ ]:
bd.databases

In [ ]:
ei_clca = bd.Database('ecoinvent310clca')
ei_bio = bd.Database('ecoinvent-3.10-biosphere')
db_template = bd.Database('bw25_db')
el_bio3 = bd.Database('biosphere3')

In [ ]:
# We can now create a new database with prospective data using the NewDatabase() function from premise

ndb = NewDatabase(
    scenarios=[
        {"model":"image", "pathway":"SSP2-RCP19", "year":2050},
        {"model":"remind", "pathway":"SSP2-PkBudg500", "year":2050},
    ],
    source_db="ecoinvent-3.10-cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.10", # <-- version of ecoinvent. Can be "3.8", "3.9" or "3.10". Must be a string.
    key='xxxxxxxxxxxxxxxxxxxxxxxxx', # <-- decryption key
    # to be requested from the library maintainers if you want ot use default scenarios included in `premise`
    keep_source_db_uncertainty=False, # False by default, set to True if you want to keep ecoinvent's uncertainty data
    keep_imports_uncertainty=False, # False by default, set to True if you want to keep the uncertainty data of the additional inventories
    use_absolute_efficiency=True, # False by default, set to True if you want to use the IAM's absolute efficiencies
)

In [ ]:
# Here is a list of all arguments that can be passed to NewDatabase():
# One task is to provide background information in .md file directly here to understand what each argument means and how the parameters can be used

scenarios: List[dict], # list of scenarios to process
source_version: str = "3.10", # version of ecoinvent database. Can be "3.6", "3.7", "3.8", "3.9" or "3.10".
source_type: str = "brightway", # type of source database. Can be "brightway" or "ecospold".
key: bytes = None, # decryption key
source_db: str = None, # name of database if source_type is "brightway"
source_file_path: str = None, # path to ecospold files if source_type is "ecospold"
additional_inventories: List[dict] = None, # list of additional inventories to import
system_model: str = "cutoff", # system model. Can be "cutoff" or "consequential".
system_args: dict = None, # arguments for the "consequential "system model
use_cached_inventories: bool = True, # use cached inventories
use_cached_database: bool = True, # use cached database
quiet=False, # suppress output
keep_imports_uncertainty=False, # keep uncertainty data of additional inventories
keep_source_db_uncertainty=False, # keep uncertainty data of source database
gains_scenario="CLE", # Air pollutionn GAINS scenario
use_absolute_efficiency=False, # use IAM's absolute efficiencies instead of efficiencies relative to 2020
biosphere_name: str = "biosphere3", # name of biosphere database in brightway project if different from "biosphere3"

In [ ]:
# In this fashion, we can create a new database with prospective data
sc = Scenario(name="my_scenario", model="remind")
sc.extract_from_brightway("ecoinvent 3.9_cutoff")


In [ ]:
sc.transform(scenario_name="SSP2-Base", year=2030)

In [ ]:
# taken from Martis IKEA branch

config = Config()
bd.projects.set_current(config.PROJECT_NAME_bw25)

# premise key
secrets_path = config.IKEA_DIR_PATH / "secrets.toml"
with open(secrets_path, "rb") as f:
secrets = tomllib.load(f)
premise_key = secrets["premise_key"]

scenarios = [
{"model": "image", "pathway": "SSP2-RCP26", "year": 2024},
{"model": "image", "pathway": "SSP2-RCP26", "year": 2025},
{"model": "image", "pathway": "SSP2-RCP26", "year": 2026},
{"model": "image", "pathway": "SSP2-RCP26", "year": 2027},
{"model": "image", "pathway": "SSP2-RCP26", "year": 2028},
{"model": "image", "pathway": "SSP2-RCP26", "year": 2029},
{"model": "image", "pathway": "SSP2-RCP26", "year": 2030},
]

ndb = premise.NewDatabase(
scenarios=scenarios,
source_db=config.ECOINVENT_ALCA_NAME,
source_version="3.10",
key=premise_key,
keep_source_db_uncertainty=False,
keep_imports_uncertainty=True,
use_absolute_efficiency=True,
system_model="cutoff",
biosphere_name=bd.config.biosphere,
)

ndb.update()  # update all sectors

ndb.write_db_to_brightway()